In [1]:
import pandas as pd
import pyodbc
import useful_functions as use
from functools import reduce

In [2]:
september_correct = pd.read_excel('../../Data/ValidationSurvey/VALIDATIONS_RETURN_092921.xlsx', sheet_name='ValidatedCorrect')
september_incorrect = pd.read_excel('../../Data/ValidationSurvey/VALIDATIONS_RETURN_092921.xlsx', sheet_name='ValidatedIncorrect')
september_not = pd.read_excel('../../Data/ValidationSurvey/VALIDATIONS_RETURN_092921.xlsx', sheet_name='NotValidated')
september_un = pd.read_excel('../../Data/ValidationSurvey/VALIDATIONS_RETURN_092921.xlsx', sheet_name='Unfinalized')
survey = pd.concat([september_correct, september_incorrect, september_not, september_un])
survey['ME'] = use.fix_me(survey.ME_NO)
survey['IQVIA_ME'] = [x[0:10] for x in survey.ME]
survey['ORIGINAL_PHONE'] = [use.fix_phone(x) for x in survey.ORIGINAL_PHONE]

In [3]:
#dhc
dhc = pd.read_csv('../../Data/DHC/DHC_2021-11.csv')
dhc = dhc.fillna('None')
dhc['ME'] = use.fix_me(dhc.ME)
dhc['DHC_PHONE'] = [use.fix_phone(x) for x in dhc['Phone_Number']]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
#data.gov
gov = pd.read_csv('../../Data/DataGov/All_Data_2021-11-09.csv')
gov = gov.fillna('None')
gov['ME'] = use.fix_me(gov.ME)
gov['GOV_PHONE'] = [use.fix_phone(x) for x in gov['phn_numbr']]

In [5]:
#iqvia
username = 'vigrose'
password = 'Ravenclaw~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        B.PHONE,
        P.ME,
        T.AFFIL_TYPE_DESC,
        A.AFFIL_IND,
        A.AFFIL_RANK
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,PHONE,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,2519493400,0010200001,Admitting,A,6
1,7275885200,0010200001,Admitting,A,6
2,8282131111,0010200003,Admitting,A,6
3,6235244000,0010200004,Attending,P,10
4,5048423000,0010200005,IDN Affiliated (Inferred),P,9


In [6]:
iqvia =iqvia.fillna('None')
iqvia['IQVIA_PHONE'] = [use.fix_phone(x) for x in iqvia['PHONE']]

In [7]:
#symphony
sym_query = \
        """
        SELECT
        d.ADDR_FRST_TLPHN_NBR AS TELEPHONE,
        l.OTHER_ID AS SYM_ME
        FROM
        ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT d, ODS.PRACTITIONER_ADDL_IDS_LAYOUT l
        WHERE
        d.DS_PRCTR_ID = l.DS_PRCTR_ID
        and
        l.ID_QLFR_TYP_CDE = 38
        """  
symphony = pd.read_sql(con=ODS, sql=sym_query)
symphony.head()

,TELEPHONE,SYM_ME
0,(858) 605-7808,0410986047
1,(503) 249-3434,0410987180
2,(361) 885-0010,0411282092
3,(401) 845-1525,0411283093
4,(202) 865-6611,0411366053


In [8]:
symphony =symphony.fillna('None')
symphony['SYM_PHONE'] = [use.fix_phone(x) for x in symphony['TELEPHONE']]

In [20]:
THIS = pd.merge(survey, symphony, left_on='IQVIA_ME', right_on='SYM_ME', how='left')
THIS = pd.merge(THIS, iqvia, left_on='IQVIA_ME', right_on='ME', how='left', suffixes = ['','_iqvia'])
THIS = pd.merge(THIS, dhc, on='ME', how='left')
THIS = pd.merge(THIS, gov, on='ME', how='left')[['ME','IQVIA_ME', 'ORIGINAL_PHONE','SYM_PHONE','IQVIA_PHONE', 'DHC_PHONE', 'GOV_PHONE', 'CORRECT_PHONE','REASON_PHONE_INCORRECT','REASON_PHONE_OTHER','CAPTURED_NUMBER','RESULT_OF_CALL']].drop_duplicates()
THIS = THIS.fillna('None')

In [27]:
dict_list = []
for row in THIS.itertuples():
    count = 0
    MATCHES = ''
    phone = row.ORIGINAL_PHONE
    symph_phone = row.SYM_PHONE
    dhc_phone = row.DHC_PHONE
    iqvia_phone = row.IQVIA_PHONE
    data_phone = row.GOV_PHONE
    total = 4-[symph_phone, dhc_phone, iqvia_phone, data_phone].count('None')
    if symph_phone == phone:
        count += 1
        MATCHES += 'Symphony, '
    if dhc_phone == phone:
        count += 1
        MATCHES += 'DHC, '
    if iqvia_phone == phone:
        count += 1
        MATCHES += 'IQVia, '
    if data_phone == phone:
        count += 1
        MATCHES += 'DataGov, '
    if total!=0:
        score = count/total
    else:
        score = 'None'
    if phone!='None':
        dicto = {
        'ME': row.ME,
        'IQVIA_ME': row.IQVIA_ME,
        'PHONE_SURVEY': phone,
        'CORRECT': row.CORRECT_PHONE,
        'CAPTURED_NUMBER':row.CAPTURED_NUMBER,
        'RESULT_OF_CALL':row.RESULT_OF_CALL,
        'MATCHED': count,
        'MATCHES': MATCHES[:-2],
        'REASON_PHONE_INCORRECT':row.REASON_PHONE_INCORRECT,
        'REASON_PHONE_OTHER':row.REASON_PHONE_OTHER,
        'TOTAL':total,
        'SCORE': score
    }
        dict_list.append(dicto)

In [28]:
phone_matches = pd.DataFrame(dict_list)

In [29]:
phone_matches = phone_matches.sort_values('MATCHED', ascending=False).drop_duplicates('ME')

In [24]:
phone_matches.groupby(['RESULT_OF_CALL']).mean()[['MATCHED']]

,MATCHED
RESULT_OF_CALL,
Changed Number,0.875000
Complete,1.200165
Contact Too Busy,1.500000
Deceased,1.000000
Direct No,1.357143
Disconnect,0.728938
Do Not Call,0.750000
Fax Modem,0.000000
Office Closed Message,0.500000


In [25]:
phone_matches.groupby(['MATCHES','RESULT_OF_CALL']).count()[['ME']]

ME
MATCHES                  RESULT_OF_CALL              
                         Changed Number             4
                         Complete                 397
                         Contact Too Busy           3
                         Direct No                  8
                         Disconnect               150
...                                               ...
Symphony, IQVia          Direct No                  2
                         Disconnect                 2
                         Other No Contact Reason    2
Symphony, IQVia, DataGov Complete                   5
                         Other No Contact Reason    2

[74 rows x 1 columns]

In [26]:
with pd.ExcelWriter(f'../../Data/ValidationSurvey/September_Triangulation.xlsx') as writer:  
        phone_matches.to_excel(writer, sheet_name='Results')
        phone_matches.groupby('MATCHES').count()[['ME']].to_excel(writer, sheet_name='All Matches')
        phone_matches.groupby('MATCHED').count()[['ME']].to_excel(writer, sheet_name='Match Count')
        phone_matches.groupby(['MATCHES','RESULT_OF_CALL']).count()[['ME']].to_excel(writer, sheet_name='Matches and Result of Call')
        phone_matches.groupby(['MATCHES','CORRECT']).count()[['ME']].to_excel(writer, sheet_name='Matches and Correctness')
        phone_matches.groupby(['RESULT_OF_CALL','MATCHED']).count()[['ME']].to_excel(writer, sheet_name='Result of Call - Matched')
        phone_matches.groupby(['CORRECT','MATCHED']).count()[['ME']].to_excel(writer, sheet_name='Correctness - Matched')
        phone_matches.groupby(['CORRECT']).mean()[['MATCHED']].to_excel(writer, sheet_name='Average # of Matches')

In [30]:
phone_matches.to_csv('../../Data/ValidationSurvey/September_eek.csv', index=False)

In [32]:
phone_matches.groupby('SCORE').count()

,ME,IQVIA_ME,PHONE_SURVEY,CORRECT,CAPTURED_NUMBER,RESULT_OF_CALL,MATCHED,MATCHES,REASON_PHONE_INCORRECT,REASON_PHONE_OTHER,TOTAL
SCORE,,,,,,,,,,,
0.000000,854,854,854,854,854,854,854,854,854,854,854
0.250000,268,268,268,268,268,268,268,268,268,268,268
0.333333,196,196,196,196,196,196,196,196,196,196,196
0.500000,373,373,373,373,373,373,373,373,373,373,373
0.666667,205,205,205,205,205,205,205,205,205,205,205
0.750000,154,154,154,154,154,154,154,154,154,154,154
1.000000,125,125,125,125,125,125,125,125,125,125,125


In [33]:
phone_matches.groupby('CORRECT').mean()

,MATCHED,TOTAL,SCORE
CORRECT,,,
N,0.629454,3.149644,0.210610
None,0.868201,3.321130,0.265952
Y,1.497494,3.443609,0.442460


In [34]:
phone_matches.groupby(['CORRECT','RESULT_OF_CALL']).mean()

MATCHED     TOTAL     SCORE
CORRECT RESULT_OF_CALL                                       
N       Complete                 0.630072  3.147971  0.211018
        Direct No                0.000000  3.000000  0.000000
        Disconnect               1.000000  4.000000  0.250000
None    Changed Number           0.875000  3.250000  0.260417
        Contact Too Busy         1.500000  3.600000  0.416667
        Deceased                 1.000000  2.000000  0.500000
        Direct No                1.440000  3.680000  0.390000
        Disconnect               0.727941  3.257353  0.227941
        Do Not Call              0.750000  3.750000  0.187500
        Fax Modem                0.000000  4.000000  0.000000
        Office Closed Message    0.500000  3.200000  0.150000
        Other No Contact Reason  0.937605  3.344013  0.287381
        Ring No Answer           0.350000  2.950000  0.129167
        Wrong Number             0.166667  3.333333  0.041667
Y       Complete                 1.500629  3.444025  0.443187
        Direct No                1.000000  3.000000  0.375000
        Wrong Number             0.000000  4.000000  0.000000